In [183]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [198]:
housing_data = pd.read_csv('housing.csv')

In [199]:
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [200]:
housing_data.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [201]:
y_val = housing_data['median_house_value']

In [202]:
x_val = housing_data.drop(['median_house_value','longitude','latitude','ocean_proximity'], axis=1)

In [203]:
x_val.head()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,41.0,880.0,129.0,322.0,126.0,8.3252
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014
2,52.0,1467.0,190.0,496.0,177.0,7.2574
3,52.0,1274.0,235.0,558.0,219.0,5.6431
4,52.0,1627.0,280.0,565.0,259.0,3.8462


In [204]:
x_val = x_val.fillna(x_val.mean())

In [205]:
X_train, X_test, y_train, y_test = train_test_split(x_val, y_val, test_size=0.3, random_state=101)

In [206]:
scaler = MinMaxScaler()

In [207]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [208]:
X_train.isnull().count()

housing_median_age    14448
total_rooms           14448
total_bedrooms        14448
population            14448
households            14448
median_income         14448
dtype: int64

In [209]:
pd.DataFrame(X_train).fillna(X_train.mean)

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
6761,19.0,2742.0,756.0,1396.0,703.0,2.5663
3010,32.0,444.0,102.0,242.0,87.0,1.1528
7812,35.0,994.0,203.0,602.0,185.0,3.5865
8480,35.0,1281.0,219.0,710.0,184.0,4.8304
1051,16.0,1257.0,231.0,559.0,213.0,4.4531
16312,32.0,4474.0,929.0,2177.0,884.0,3.2889
2042,23.0,2097.0,385.0,911.0,405.0,3.5128
1755,46.0,1901.0,295.0,833.0,352.0,5.5196
16022,52.0,2401.0,346.0,812.0,328.0,6.8322
20441,24.0,4072.0,582.0,1834.0,565.0,6.0487


In [210]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))